Họ tên: Đàm Tử Tâm\
MSSV: 21120551\
Lớp: 21_2 (chính quy)\
Môn: Toán ứng dụng & thống kê

# Đồ án 5:  Linear Regression

#### Nội dung đồ án:
File "wine.csv" là cơ sở dữ liệu đánh giá chất lượng của 1200 chai rượu vang theo thang điểm 1 - 10 dựa trên 11 tính chất khác nhau.

Xây dựng mô hình đánh giá chất lượng rượu sử dụng phương pháp hồi quy tuyến tính.\
a. Sử dụng toàn bộ 11 đặc trưng đề bài cung cấp.\
b. Sử dụng duy nhất 1 đặc trưng cho kết quả tốt nhất. (Gợi ý: Phương pháp Cross Validation)\
c. Xây dựng một mô hình của riêng bạn cho kết quả tốt nhất.

Lưu ý: sinh viên phải sử dụng thuật toán OLS đã được hướng dẫn trong phần bài tập. Sinh viên
không được dùng các hàm có sẵn của các thư viện để thực hiện các tính toán.

#### Quy định bài nộp
- Thực hiện toàn bộ bài làm trên 1 tập tin Jupyter Notebook (.ipynb)
- Đầu bài phải có phần trình bày thông tin sinh viên và giải thuật.
- Cuối bài phải có phần mô tả ý tưởng thực hiện và mô tả các hàm

#### Quy định chấm bài
Những trường hợp sau đây sẽ bị 0 điểm toàn bộ đồ án:
- Nộp sai quy định bài nộp
- Thực thi mã nguồn báo lỗi

LƯU Ý: SAO CHÉP BÀI LÀM CỦA NHAU SẼ BỊ 0 ĐIỂM TOÀN BỘ PHẦN THỰC HÀNH

# Giải thuật OLS

OLS (Ordinary Least Squares) là một thuật toán được sử dụng trong việc tìm mô hình tuyến tính tốt nhất để dự đoán một biến phụ thuộc dựa trên các biến độc lập. Thuật toán này cố gắng tìm cách tối thiểu hóa sai số giữa giá trị dự đoán và giá trị thực tế thông qua việc tìm các hệ số tối ưu cho các biến độc lập.

Chi tiết thuật toán OLS như sau:

1. Chuẩn bị dữ liệu:

Xác định biến phụ thuộc (y) và các biến độc lập (x1, x2, ..., xn).
Tạo ma trận thiết kế X với mỗi hàng là một quan sát và mỗi cột là giá trị của các biến độc lập, bao gồm cột đơn vị 1 để tính toán hệ số chặn.
Tạo vector y chứa giá trị của biến phụ thuộc tương ứng với mỗi quan sát.

2. Tính toán hệ số:

Sử dụng công thức sau: β = $(X^TX)^{-1}(X^Ty)$.
Trong đó, β là vector chứa các hệ số ước lượng cho các biến độc lập, $X^T$ là ma trận chuyển vị của $X$ và $(X^TX)^{-1}$ là ma trận nghịch đảo của tích $X^T$ và $X$.

3. Dự đoán giá trị:

Khi có các hệ số ước lượng β, ta có thể dự đoán giá trị biến phụ thuộc y dựa trên các giá trị của biến độc lập bằng công thức: y_pred = X * β.

4. Đánh giá mô hình:

Để đánh giá mô hình, ta có thể sử dụng các thước đo như Sai số bình phương trung bình (MSE), Sai số trung bình tuyệt đối (MAE), $R^2$ (hệ số xác định mô hình), và kiểm tra giả thuyết về các hệ số (như kiểm định t-Test).

5. Lựa chọn mô hình:

Dựa trên đánh giá mô hình và mục tiêu cụ thể, ta có thể điều chỉnh các biến độc lập, thêm hoặc loại bỏ các biến để tạo ra mô hình tốt hơn.
Thuật toán OLS là một phương pháp mạnh mẽ để tìm mô hình tuyến tính tốt nhất cho dữ liệu. Tuy nhiên, nó có một số giả định như mối quan hệ tuyến tính giữa biến phụ thuộc và biến độc lập, không tồn tại đa cộng tuyến (multicollinearity) mạnh giữa các biến độc lập và không có sự sai biệt hệ thống (systematic bias) trong các quan sát. Điều này cần được xem xét khi áp dụng thuật toán OLS vào bài toán cụ thể.

In [44]:
import pandas as pd
df = pd.read_csv('C:/Users/this pc/Desktop/wine.csv')

data = df.values.tolist()
#data=[[2,1],[2,2],[5,3],[8,4]]

def deepcopy_matrix(A):
    C=[]
    for i in range(len(A)):
        temp=[]
        for j in range(len(A[i])):
            temp.append(A[i][j]*float(1))
        C.append(temp)
    return C

def create_identity_matrix(n):
    A=[]
    for i in range(0,n):
        temp=[]
        for j in range(0,n):
            if(i==j):
                temp.append(1.0)
            else:
                temp.append(0.0)            
        A.append(temp)
    return A
def inverse(A1):
    
    A=deepcopy_matrix(A1)
    
    #kiem tra neu ma tran A input khong hop le
    for i in range(0,len(A)):
        if len(A) != len(A[i]):
            print("Ma tran input khong vuong, khong the tim ma tran nghich dao!")
            return   
    
    #tao ma tran In
    B=create_identity_matrix(len(A))
        
    
    flag=True
    k=0
    j=0
    
    ##duyet tung cot
    for i in range(0,len(A[0])):
        k=j
            
        flag=False
        
        #duyet tung dong cua cot do de kiem tra cot do co = 0 hay ko ?
        for l in range(j,len(A)):
            if A[l][i] != 0:
                flag=True
                
        if flag == True:
            if A[k][i] == 0:
                for z in (k+1,len(A)):
                    if A[z][i] != 0:
                        temp=A[k]
                        A[k]=A[z]
                        A[z]=temp
                        
                        temp=B[k] #cua B
                        B[k]=B[z] #cua B
                        B[z]=temp #cua B
                        break
            d=(float(1)/A[k][i])
            for m in range(0,len(A[0])):
                A[k][m]=A[k][m]*float(d)
                B[k][m]=B[k][m]*float(d)
            
            for k1 in range (0,len(A)):
                if k1==i:
                    continue
                d=-A[k1][i]/float(A[k][i])
                for m in range(0,len(A[0])):
                    A[k1][m]=A[k1][m]+float(d)*A[k][m]
                    B[k1][m]=B[k1][m]+float(d)*B[k][m]
            j=j+1
        else:
            print("Ma tran khong kha nghich")
            return
    return B

def multi_matrix(A,B):
    if(len(A[0])!=len(B)):
        print("Khong the nhan hai ma tran")
        return None
    C=[]
    temp=[]
    ans=0.0
    for i in range(len(A)):
        temp=[]
        for j in range(len(B[0])):
            ans=0.0
            for k in range(len(A[0])):
                ans+=A[i][k] * float(B[k][j])
            temp.append(ans)
        C.append(temp)
    return C

def transpose_matrix(A):
    C=[]
    for i in range(len(A[0])):
        temp=[]
        for j in range(len(A)):
            temp.append(A[j][i])
        C.append(temp)
    return C

#---------------------------------

def linear_B(data):
    A=[]
    for i in range(len(data)):
        temp=[]
        temp.append(1.0)
        for j in range(1,len(data[i])):
            temp.append(data[i][j])
        A.append(temp)


    Y=[]
    for i in range(len(data)):
        temp=[]
        temp.append(data[i][0])
        Y.append(temp)

    AT=transpose_matrix(A)
    ATA1=multi_matrix(AT,A)

    ATA1=inverse(ATA1)
    ATY=multi_matrix(AT,Y)
    B=multi_matrix(ATA1,ATY)
    return B
    
def linear_model(B):

    ans='f(' #x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11) = '

    var=[]
    for i in range(1,len(B)):
        var.append('x'+ str(i))
    for i in range(0,len(var)):
        ans+=var[i]
        if i != (len(var)-1):
            ans+=','
    ans+=') = '             
        #---------------
    for i in range(len(B)):
        if i==0:
            ans+=str(B[i][0])+' '
        else:
            ans+=str(B[i][0])+'*'+var[i-1]+' '
        if i!=(len(B)-1):
            if(B[i+1][0] >=0):
                ans+='+'
    return ans

def solve_linear_module(B,data):
    ans=[]    
    sum=0.0
    for k in range(len(data)):
        temp=[]
        sum=0.0
        for i in range(len(B)):
            if i==0:
                sum+=B[i][0]
            else:
                sum+=B[i][0]*float(data[k][i])
        temp.append(sum)
        ans.append(temp)
        
    return ans

def find_r(data):    
    B=linear_B(data)
    Y=[]
    for i in range(len(data)):
        temp=[]
        temp.append(data[i][0])
        Y.append(temp)    
    Y1=solve_linear_module(B,data)   
    
    r=[]
    for i in range(len(Y)):
        r.append(Y[i][0]-Y1[i][0])
    ans=0.0
    for i in range(len(r)):
        ans+=float(r[i])*r[i]
        
    return (ans**(0.5))

def findmin_returnindex(l):
    min=l[0]
    index=0
    for i in range(len(l)):
        if(l[i] < min):
            min=l[i]
            index=i
    return index

def print_bestmodule_1var(data,df):
    
    A1=[]
    for i in range(len(data)):
        temp=[]
        temp.append(data[i][0])
        A1.append(temp)
    
    Cl=[]
    rl=[]
    for i in range(1,len(data[0])):
        A=deepcopy_matrix(A1)
        for j in range (len(data)):
            A[j].append(data[j][i])
            
        print('Mô hình '+str(i)+': dựa trên đặc trưng '+str(df.columns[i]))
        B=linear_B(A)
        C=linear_model(B)
        print(C)
        print('Chuẩn r = ',end='')
        r=find_r(A)
        print(r)
        print('')
        
        Cl.append(C)
        rl.append(r)
    index=findmin_returnindex(rl)
    print('KẾT LUẬN: mô hình cho ra kết quả tốt nhất (chuẩn thấp nhất): '+Cl[index]+' khi dựa trên đặc trưng '+str(df.columns[index+1]))
    print('Chuẩn r = '+str(rl[index]))
    return

#---------------------cau c-----------------

def print_bestmodule_custom(data,df):
    
    
    
    Cl=[]
    rl=[]
    for i in range(1,len(data[0])):
        dropname=str(df.columns[i])    
        df2 = df.drop(df.columns[[ i]],axis = 1)
        data2= df2.values.tolist()
            
        print('Mô hình '+str(i)+': không sử dụng đặc trưng '+dropname)
        B=linear_B(data2)
        C=linear_model(B)
        print(C)
        print('Chuẩn r = ',end='')
        r=find_r(data2)
        print(r)
        print('')
        
        Cl.append(C)
        rl.append(r)
    index=findmin_returnindex(rl)
    print('KẾT LUẬN: mô hình cho ra kết quả tốt nhất (chuẩn thấp nhất): '+Cl[index]+' khi không sử dụng đặc trưng '+str(df.columns[index+1]))
    print('Chuẩn r = '+str(rl[index]))
    return


#--------------------------test-----------------------   
    

print('----------------------------------- MÔ HÌNH ĐÁNH GIÁ CHẤT LƯỢNG RƯỢU -----------------------------------')

print('')
print('')
print('[ A - MÔ HÌNH SỬ DỤNG PHƯƠNG PHÁP HỒI QUY TUYẾN TÍNH (SỬ DỤNG TOÀN BỘ 11 ĐẶC TRƯNG CỦA RƯỢU) ]')
print('')

B=linear_B(data)
C=linear_model(B)
print(C)
print('Chuẩn r = ',end='')
r=find_r(data)
print(r)

print('')
print('')
print('[ B - MÔ HÌNH SỬ DỤNG PHƯƠNG PHÁP HỒI QUY TUYẾN TÍNH (SỬ DỤNG DUY NHẤT 1 ĐẶC TRƯNG CỦA RƯỢU CHO RA KẾT QUẢ TỐT NHẤT) ]')
print('')
print_bestmodule_1var(data,df)



print('')
print('')
print('[ C - XÂY DỰNG MÔ HÌNH CÁ NHÂN ]')
print('Mỗi mô hình đánh giá chất lượng rượu sử dụng phương pháp hồi quy tuyến tính dưới đây đều bỏ đi 1/11 đặc trưng có của rượu để xây dựng. Từ đó tìm ra mô hình có chuẩn nhỏ nhất sẽ là mô hình tốt nhất khi xây dựng bỏ đi đặc trưng tương ứng.')
print('')

print_bestmodule_custom(data,df)

print('')
print('Nhận xét: các mô hình cá nhân tự xây dựng đều không tốt bẳng mô hình sử dụng cả 11 đặc trưng. Tuy nhiên, sau nhiều mô hình thử nghiệm thì rút ra được mô hình ở kết luận phía trên là mô hình cá nhân tốt nhất.')


----------------------------------- MÔ HÌNH ĐÁNH GIÁ CHẤT LƯỢNG RƯỢU -----------------------------------


[ A - MÔ HÌNH SỬ DỤNG PHƯƠNG PHÁP HỒI QUY TUYẾN TÍNH (SỬ DỤNG TOÀN BỘ 11 ĐẶC TRƯNG CỦA RƯỢU) ]

f(x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11) = 42.91716228888254 +0.047524752892812216*x1 -1.0687425805320174*x2 -0.26871082850604466*x3 +0.03497426610914545*x4 -1.5972956049074014*x5 +0.0034878813769961547*x6 -0.0037983550575281244*x7 -39.46908086675103*x8 -0.24557590904441895*x9 +0.7738407941094749*x10 +0.26937749584209314*x11 
Chuẩn r = 22.09471680779166


[ B - MÔ HÌNH SỬ DỤNG PHƯƠNG PHÁP HỒI QUY TUYẾN TÍNH (SỬ DỤNG DUY NHẤT 1 ĐẶC TRƯNG CỦA RƯỢU CHO RA KẾT QUẢ TỐT NHẤT) ]

Mô hình 1: dựa trên đặc trưng fixed acidity
f(x1) = 5.0806344171184605 +0.06771800901930902*x1 
Chuẩn r = 27.70877660655169

Mô hình 2: dựa trên đặc trưng "volatile acidity"
f(x1) = 6.555645007462523 -1.7161788508893494*x1 
Chuẩn r = 25.92104625750291

Mô hình 3: dựa trên đặc trưng "citric acid"
f(x1) = 5.389255872040120

# Mô tả, ý tưởng thực hiện các hàm

**Quy ước (1):** trong đồ án này, các dữ liệu trong ma trận/dataframe được quy ước cột đầu tiên là biến phụ thuộc, các cột còn lại là biến độc lập.\
**Quy ước (2):** vector chứa các hệ số ước lượng của mô hình hồi quy tuyến tính luôn có phần tử đầu tiên là hệ số chặn, các phần tử còn lại là hệ số góc.

Đầu tiên, đọc file wine.csv vào dataframe df bằng thư viện pandas, sau đó chuyển dataframe df thành ma trận bằng hàm values.tolist() và gán vào data (để dễ dàng truy xuất các phần tử sau này).

**1. Hàm deepcopy_matrix:** dùng để sao chép ma trận input ra một ma trận mới và trả về ma trận này trong output, dựa trên ý tưởng của hàm deepcopy trong thư viện copy.

**2. Hàm create_identity_matrix:** dùng để tạo và trả về ma trận đơn vị cấp n. Dùng hai vòng for lồng nhau để lần lượt thêm các giá trị 0 vào ma trận, trừ các phần tử trên đường chéo chính thì giá trị sẽ là 1

**3. Hàm inverse:** dùng để tìm và trả về ma trận nghịch đảo của một ma trận vuông cấp n
- B1: Đầu tiên, kiểm tra ma trận đầu vào có vuông hay không, nếu không vuông thì ngưng hàm
- B2: Nếu ma trận input là ma trận vuông cấp n, tiến hành khởi tạo một ma trận đơn vị B cấp n bằng hàm create_identity_matrix
- B3: Duyệt qua từng cột của ma trận input, nếu cột đang xét có phần tử nằm từ đường chéo chính trở xuống đều bằng 0 thì ngưng hàm, thông báo ma trận không khả nghịch, ngược lại thì thực hiện các bước sau:
    + B3.1: Kiểm tra nếu phần tử nằm trên đường chéo chính bằng không thì hoán vị dòng chứa phần tử này với dòng có phần tử ở cột tương ứng khác 0 gần nhất với dòng đang xét. Việc này được thực hiện cùng lúc ở ma trận input và ma trận đơn vị B đã tạo ra trước đó.
    + B3.2: Nhân dòng đang xét của ma trận input và dòng tương ứng của ma trận đơn vị B với 1/(phần tử ở đường chéo chính của ma trận input).
    + B3.3: Cộng các dòng còn lại của ma trận input và cả ma trận đơn vị B (trừ dòng chứa phần tử nằm trên đường chéo chính) với một số thích hợp để biến các phần tử ở cột đang xét của ma trận input đều bằng 0.
    + B3.4: Lặp lại B3
- B4: Ma trận đơn vị B được tạo ra ở B2 sau khi thực hiện cả 3 bước trên sẽ là ma trận nghịch đảo của ma trận input, hàm sẽ trả về ma trận B

**4. Hàm multi_matrix:** nhân hai ma trận input (A x B) và trả về ma trận kết quả trong output. Đầu tiên kiểm tra nếu số cột của ma trận A khác số dòng của ma trận B thì in ra màn hình không thể nhân hai ma trận và kết thúc hàm. Ngược lại, dùng 3 vòng lặp for lồng nhau để nhân hai ma trận: xét từ vòng for ngoài cùng vào, vòng for i lần lượt duyệt qua từng dòng của A, rồi vòng for j lồng phía trong duyệt từng cột của B và vòng for trong cùng là vòng for k duyệt từng cột của A. Trong vòng for k, lấy tổng A[i][k] x B[k][j]. Sau khi chạy hết vòng for k, ta được phần tử C[i][j] trong ma trận kết quả.

**5. Hàm transpose_matrix:** hàm thực hiện chuyển vị ma trận input và trả về ma trận đã chuyển vị. Đầu tiên, tạo vòng for i duyệt qua từng cột của ma trận input, lồng trong vòng for i là vòng for j, duyệt qua từng dòng của ma trận input. Bằng hai vòng for này, lần lượt đưa các cột của ma trận input vào các dòng của ma trận output, sau khi chạy hết hai vòng for ta được ma trận input đã chuyển vị, hàm trả về ma trận này.

**6. Hàm linear_B:** hàm nhận input là một ma trận dữ liệu data đáp ứng quy ước **(1)**, hàm sẽ trả về vector cột B chứa các hệ số ước lượng của mô hình hồi quy tuyến tính thỏa quy ước **(2)**. 
- Đầu tiên, tạo ra ma trận A với mỗi hàng là một quan sát của một đối tượng trong ma trận dữ liệu data và mỗi cột là giá trị của các biến độc lập, bao gồm cả cột đơn vị 1 đứng đầu tiên để tính toán hệ số chặn. Tạo vector Y chứa giá trị của biến phụ thuộc tương ứng với mỗi quan sát.
- Tính $A^T$ bằng hàm transpose_matrix và lưu vào AT
- Tính $A^TA$ bằng hàm multi_matrix và lưu vào ATA1
- Tính $(A^TA)^{-1}$ bằng hàm inverse và lưu lại vào ATA1
- Tính $A^TY$ bằng hàm multi_matrix và lưu vào ATY
- Tính  $(A^TA)^{-1}(A^TY)$ bằng hàm multi matrix và lưu vào B, lúc này B là vector cột chứa các hệ số ước lượng của mô hình hồi quy tuyến tính thỏa quy ước **(2)**, hàm trả về vector này

**7. Hàm linear_model:** hàm nhận input là vector cột B chứa các hệ số ước lượng của mô hình hồi quy tuyến tính thỏa quy ước **(2)**, hàm sẽ trả về mô hình hồi quy tuyến tính. Đầu tiên, tạo chuỗi ans để lưu mô hình, nếu B có n phần tử thì mô hình có n-1 biến độc lập. Đầu tiên đưa phần tử đầu tiên của B là hệ số chặn vào mô hình (ans), sau đó lần lượt đưa tích từng phần tử tiếp theo (các hệ số góc) của B với từng biến độc lập vào mô hình. Hàm trả về mô hình lưu trong ans. 

**8. Hàm solve_linear_module:** hàm nhận input là vector cột B chứa các hệ số ước lượng của mô hình hồi quy tuyến tính thỏa quy ước **(2)** và một ma trận dữ liệu data thỏa quy ước **(1)**, hàm tính và trả về giá trị dự đoán của biến phụ thuộc y dựa trên các giá trị của biến độc lập trong data bằng mô hình hồi quy tuyến tính. Ý tưởng: với mỗi dòng dữ liệu trong data (trừ cột đầu tiên vì là biến phụ thuộc), thế vào mô hình hồi quy tuyến tính có các hệ số ước lượng trong B rồi đưa từng đáp án vào vector ans. Cuối cùng, ta được vector ans là giá trị dự đoán của biến phụ thuộc y dựa trên các giá trị của biến độc lập trong data bằng mô hình hồi quy tuyến tính, hàm trả về vector này.

**9. Hàm find_r:** input là ma trận dữ liệu data thỏa quy ước **(1)**, hàm tính và trả về chuẩn của mô hình hồi quy tuyến tính. Đầu tiên, tìm hệ số ước lượng của mô hình bằng hàm linear_B và lưu vào B, sau đó dùng B để tìm các giá trị dự đoán của biến phụ thuộc y bằng hàm solve_linear_module, lưu vào Y1. Đồng thời, đưa cột đầu tiên của ma trận data (giá trị của biến phụ thuộc) vào Y. Sau đó, tính chuẩn bằng công thức $r=\sqrt{(Y-Y1)^2}$ và trả về chuẩn r của mô hình.

**10. Hàm findmin_returnindex:** hàm trả về vị trí chứa giá trị nhỏ nhất trong một list.

**11. Hàm print_bestmodule_1var:** hàm in ra mô hình đánh giá chất lượng rượu bằng phương pháp hồi quy tuyến tính sử dụng duy nhất 1 đặc trưng cho kết quả tốt nhất. Input là ma trận dữ liệu data và dataframe df thỏa quy ước **(1)**, lưu ý dataframe chỉ dùng để trích xuất tên của các đặc trưng sử dụng trong mô hình. Đầu tiên, hàm đưa cột đầu tiên của data (biến phụ thuộc) vào A1. Sau đó, với mỗi cột dữ liệu tiếp theo trong data, hàm sẽ copy một ma trận A1 và gán vào A, đưa cột dữ liệu tiếp theo vào A rồi tính và in ra mô hình và chuẩn của mô hình dựa trên đặc trưng tương ứng. Sau khi tính hết dựa trên từng đặc trưng, hàm tìm mô hình có chuẩn nhỏ nhất và in ra, đây sẽ là mô hình tốt nhất.

**12. Hàm print_bestmodule_custom:** hàm in ra các mô hình cá nhân đánh giá chất lượng rượu bằng phương pháp hồi quy tuyến tính sử dụng 10/11 đặc trưng và kết luận mô hình cho ra kết quả tốt nhất. Input là ma trận dữ liệu data và dataframe df thỏa quy ước **(1)**, lưu ý dataframe chỉ dùng để trích xuất tên của đặc trưng bị bỏ đi trong mô hình trong mỗi lần thử. Mỗi mô hình đánh giá chất lượng rượu sử dụng phương pháp hồi quy tuyến tính dưới đây đều bỏ đi 1/11 đặc trưng có của rượu để xây dựng. Từ đó tìm ra mô hình có chuẩn nhỏ nhất sẽ là mô hình tốt nhất khi xây dựng bỏ đi đặc trưng tương ứng.